In [ ]:
import py_eddy_tracker_sample
from py_eddy_tracker.generic import flatten_line_matrix
from py_eddy_tracker.observations.tracking import TrackEddiesObservations
import numba as nb
import holoviews as hv
import panel as pn
import numpy as np

hv.extension('bokeh', logo=False)
pn.extension()


# essai dans une classe

In [ ]:
class MyTrackEddiesObservations(TrackEddiesObservations):
    def visu(self, contour_type = "speed"):
        if contour_type == "speed":
            contour_lon = self.contour_lon_s
            contour_lat = self.contour_lat_s
            
        elif contour_type == "effective":
            contour_lon = self.contour_lon_e
            contour_lat = self.contour_lat_e
            
        else:
            print("bad contour_type")
            return None
        
        full_time = self.time
        timed = np.unique(full_time)

        slider = pn.widgets.DiscretePlayer(name="date", options=list(timed), value=timed[0])

        def select(tn, lon, lat):
            mask = full_time == tn

            sub_lon = lon[mask]
            sub_lat = lat[mask]

            _lon = flatten_line_matrix(sub_lon)
            _lat = flatten_line_matrix(sub_lat)
            return np.array([_lon, _lat]).T

        extents = (-4, 34, 30, 45)

        @pn.depends(slider)
        def update(*args, **kwargs):
            print("args = ", args)
            print("kwargs = ", kwargs)
            date = kwargs["value"]
            return hv.Path([select(date, contour_lon, contour_lat)])

        @pn.depends(slider)
        def plot_date(value):
            return pn.pane.HTML(f"<h1>date = {np.datetime64('1950-01-01')+np.timedelta64(value)}</h1>")
        return pn.Column(
            pn.Row(slider, plot_date),
            (       
                 hv.DynamicMap(update)
            ).opts(width=900, height=600)
        )

c = MyTrackEddiesObservations.load_file(
    py_eddy_tracker_sample.get_demo_path("eddies_med_adt_allsat_dt2018/Cyclonic.zarr")
)
c.contour_lon_s[:] = (c.contour_lon_s + 180) % 360 -180

c.visu("speed")